In [ ]:
!pip install gradio transformers

In [ ]:
import gradio as gr
from transformers import pipeline

# --- Tải 2 LLM ---
print("Loading Model 1: Non-recovery (Llama-0.8B-mlp-embedding)")
# Khởi tạo pipeline với device_map="auto" để tận dụng GPU tốt nhất nếu có
generator1 = pipeline('text-generation', model='TheGardener/Llama-0.8B-mlp-embedding', device_map="auto")
print("Model 1 loaded.")

print("Loading Model 2: Recovery (KD-llama-0.8B-demo)")
generator2 = pipeline('text-generation', model='TheGardener/MLP-pruner-activation-llama3.2-1.13B', device_map="auto")
print("Model 2 loaded.")

# --- Định nghĩa hàm xử lý cho từng LLM ---
# Thêm tham số temperature vào các hàm này
def generate_text_with_model1(prompt, temperature):
    if not prompt:
        return "Vui lòng nhập văn bản cho Mô hình 1."
    # Truyền temperature vào hàm generator
    output = generator1(
        prompt,
        max_new_tokens=100, # Tăng max_new_tokens để thấy rõ hơn sự khác biệt
        num_return_sequences=1,
        truncation=True,
        temperature=temperature,
        do_sample=True # Cần bật do_sample=True để temperature có hiệu lực
    )[0]['generated_text']
    return output

def generate_text_with_model2(prompt, temperature):
    if not prompt:
        return "Vui lòng nhập văn bản cho Mô hình 2."
    output = generator2(
        prompt,
        max_new_tokens=100, # Tăng max_new_tokens
        num_return_sequences=1,
        truncation=True,
        temperature=temperature,
        do_sample=True # Cần bật do_sample=True để temperature có hiệu lực
    )[0]['generated_text']
    return output

# --- Tạo một giao diện chung để so sánh 2 LLM ---
def compare_llm_outputs(input_text, temperature):
    if not input_text:
        return "Vui lòng nhập văn bản để so sánh.", "Vui lòng nhập văn bản để so sánh."

    # Gọi LLM 1 (Non-recovery)
    output_model1 = generator1(
        input_text,
        max_new_tokens=100,
        num_return_sequences=1,
        truncation=True,
        temperature=temperature,
        do_sample=True
    )[0]['generated_text']

    # Gọi LLM 2 (With-recovery)
    output_model2 = generator2(
        input_text,
        max_new_tokens=100,
        num_return_sequences=1,
        truncation=True,
        temperature=temperature,
        do_sample=True
    )[0]['generated_text']

    return output_model1, output_model2

# --- Xây dựng giao diện Gradio ---
# Giao diện cho LLM 1 (Non-recovery Text Generation)
interface1 = gr.Interface(
    fn=generate_text_with_model1,
    inputs=[
        gr.Textbox(lines=5, label="Nhập Prompt cho Mô hình Non-recovery"),
        gr.Slider(minimum=0.1, maximum=1.5, value=0.7, step=0.1, label="Temperature", info="Kiểm soát tính ngẫu nhiên của output.")
    ],
    outputs=gr.Textbox(label="Văn bản được tạo (Non-recovery)"),
    title="Mô hình 1: Llama-0.8B (Non-recovery)",
    description="Mô hình này tạo văn bản dựa trên prompt của bạn. Điều chỉnh Temperature để thay đổi tính ngẫu nhiên."
)

# Giao diện cho LLM 2 (With-recovery Text Generation)
interface2 = gr.Interface(
    fn=generate_text_with_model2,
    inputs=[
        gr.Textbox(lines=5, label="Nhập Prompt cho Mô hình With-recovery"),
        gr.Slider(minimum=0.1, maximum=1.5, value=0.7, step=0.1, label="Temperature", info="Kiểm soát tính ngẫu nhiên của output.")
    ],
    outputs=gr.Textbox(label="Văn bản được tạo (With-recovery)"),
    title="Mô hình 2: Llama-0.8B (With-recovery)",
    description="Mô hình này tạo văn bản dựa trên prompt của bạn. Điều chỉnh Temperature để thay đổi tính ngẫu nhiên."
)

# Giao diện so sánh 2 LLM
comparison_interface = gr.Interface(
    fn=compare_llm_outputs,
    inputs=[
        gr.Textbox(lines=7, label="Nhập văn bản để so sánh giữa 2 mô hình"),
        gr.Slider(minimum=0.1, maximum=1.5, value=0.7, step=0.1, label="Temperature", info="Temperature áp dụng cho cả hai mô hình.")
    ],
    outputs=[
        gr.Textbox(label="Kết quả từ Mô hình 1 (Non-recovery)", show_copy_button=True),
        gr.Textbox(label="Kết quả từ Mô hình 2 (With-recovery)", show_copy_button=True)
    ],
    title="So sánh Llama-0.8B (Non-recovery vs. With-recovery)",
    description="Nhập một văn bản để xem hai mô hình Llama-0.8B tạo ra kết quả như thế nào. Điều chỉnh Temperature để thay đổi tính ngẫu nhiên của cả hai."
)

# Khởi chạy ứng dụng Gradio với các tab
app = gr.TabbedInterface([interface1, interface2, comparison_interface],
                         ["Non-recovery Model", "With-recovery Model", "So sánh 2 Mô hình"])
app.launch(share=True)

In [ ]:
import gradio as gr
from transformers import pipeline

# --- Tải 3 LLM ---
print("Loading Model 1: Non-recovery (Llama-0.8B-mlp-embedding)")
generator1 = pipeline('text-generation', model='TheGardener/Llama-0.8B-mlp-embedding', device_map="auto")
print("Model 1 loaded.")

print("Loading Model 2: Recovery (KD-llama-0.8B-demo)")
generator2 = pipeline('text-generation', model='TheGardener/KD-llama-0.8B-demo', device_map="auto")
print("Model 2 loaded.")

# Thêm Mô hình 3
# Bạn cần thay thế 'your_third_model_name_here' bằng tên mô hình cụ thể của bạn
# Ví dụ: Một phiên bản Llama-0.8B khác, hoặc một mô hình base
print("Loading Model 3: Llama-3.2-1B-teacher")
generator3 = pipeline('text-generation', model='TheGardener/Llama-3.2-1B-wikitext-finetune', device_map="auto") # Thay thế 'distilgpt2' bằng mô hình của bạn
print("Model 3 loaded.")

# --- Hàm xử lý chung để so sánh 3 LLM ---
def compare_three_llm_outputs(input_text, temperature):
    if not input_text:
        return "Vui lòng nhập văn bản để so sánh.", \
               "Vui lòng nhập văn bản để so sánh.", \
               "Vui lòng nhập văn bản để so sánh."

    # Gọi LLM 1 (Non-recovery)
    output_model1 = generator1(
        input_text,
        max_new_tokens=100,
        num_return_sequences=1,
        truncation=True,
        temperature=temperature,
        do_sample=True
    )[0]['generated_text']

    # Gọi LLM 2 (With-recovery)
    output_model2 = generator2(
        input_text,
        max_new_tokens=100,
        num_return_sequences=1,
        truncation=True,
        temperature=temperature,
        do_sample=True
    )[0]['generated_text']

    # Gọi LLM 3
    output_model3 = generator3(
        input_text,
        max_new_tokens=100,
        num_return_sequences=1,
        truncation=True,
        temperature=temperature,
        do_sample=True
    )[0]['generated_text']

    return output_model1, output_model2, output_model3

# --- Xây dựng giao diện Gradio để so sánh 3 LLM ---
comparison_interface = gr.Interface(
    fn=compare_three_llm_outputs,
    inputs=[
        gr.Textbox(lines=7, label="Nhập văn bản để so sánh giữa 3 mô hình", placeholder="Viết một câu chuyện về..."),
        gr.Slider(minimum=0.1, maximum=1.5, value=0.7, step=0.1, label="Temperature", info="Kiểm soát tính ngẫu nhiên của output (áp dụng cho cả ba mô hình).")
    ],
    outputs=[
        gr.Textbox(label="Kết quả từ Mô hình 1 (Non-recovery)", show_copy_button=True),
        gr.Textbox(label="Kết quả từ Mô hình 2 (With-recovery)", show_copy_button=True),
        gr.Textbox(label="Kết quả từ Mô hình 3 (teacher)", show_copy_button=True) # Cập nhật nhãn này
    ],
    title="So sánh 3 mô hình Llama-0.8B (Non-recovery vs. With-recovery vs. Mô hình thứ 3)",
    description="Nhập một văn bản để xem ba mô hình Llama-0.8B tạo ra kết quả như thế nào. Điều chỉnh Temperature để thay đổi tính ngẫu nhiên của các kết quả."
)

# Khởi chạy ứng dụng Gradio (chỉ với một giao diện so sánh)
comparison_interface.launch(share=True)